In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import datetime as dt
import talib as ta
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import dash
from dash import dcc, html

In [2]:
data_btc = yf.download('BTC-USD', period="10y", interval="1d")

[*********************100%***********************]  1 of 1 completed


In [3]:
data = data_btc.copy()

MACD_FAST = 12
MACD_SLOW = 26
MACD_SIGNAL = 9

close_prices = data['Close'].astype(float).values
close_prices = close_prices.ravel()  # También puedes usar .flatten()

data['macd'], data['macdSignal'], data['macdHist'] = ta.MACD(close_prices,fastperiod=MACD_FAST, slowperiod=MACD_SLOW, signalperiod=MACD_SIGNAL)
data['macd_test'] = np.where((data.macd > data.macdSignal), 1, 0)
# print(close_prices)

In [ ]:
import plotly.graph_objects as go

# Crear el gráfico interactivo con Plotly
fig = go.Figure()

# Añadir serie de precios de cierre
fig.add_trace(go.Scatter(
    x=data.index, 
    y=data['Close'], 
    mode='lines', 
    name='Precio de Cierre', 
    line=dict(color='green', width=1)
))

# # Añadir SMA de 50 períodos
# fig.add_trace(go.Scatter(
#     x=data.index, 
#     y=data['SMA_50'], 
#     mode='lines', 
#     name='SMA 50', 
#     line=dict(color='blue', width=2)
# ))

# Personalizar el diseño del gráfico
fig.update_layout(
    title=f'Precio de Cierre y SMA 50',
    xaxis_title='Fecha',
    yaxis_title='Precio',
    xaxis_rangeslider_visible=True,  # Mostrar slider de zoom
    template='plotly_dark'  # Estilo oscuro
)

# Mostrar el gráfico interactivo
fig.show()


KeyError: 'SMA_50'

In [ ]:
ma1=int(input('ma1:'))
ma2=int(input('ma2:'))
stdate=input('start date in format yyyy-mm-dd:')
eddate=input('end date in format yyyy-mm-dd:')
ticker=input('ticker:')